In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from connect_oracle import criar_conexao

In [3]:
df = pd.read_excel("superstore.xlsx", sheet_name="Orders")

In [4]:
connection = criar_conexao()
cursor = connection.cursor()

In [72]:
cursor.execute("""
    begin
        execute immediate 'drop table orders';
        exception when others then if sqlcode <> -942 then raise; end if;
    end;""")

In [73]:
cursor.execute("""
    create table orders (
               Order_ID number generated always as identity,
               Order_Code varchar2(15),
               Order_Date date,
               Customer_ID varchar2(8),
               Ship_Date date,
               Ship_Mode varchar2(20),
               Product_ID varchar2(15),
               Sales number(10,2),
               Quantity number (10,0),
               Discount number(10,2),
               Profit number(10,2),
               primary key (Order_ID))""")

In [62]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [63]:
orders = df.drop(
    columns=[
        'Row ID', 
        'Customer Name', 
        'Segment', 
        'Country', 
        'City', 
        'State', 
        'Postal Code', 
        'Region', 
        'Category', 
        'Sub-Category',
        'Product Name'])

In [75]:
orders.columns

Index(['Order_Code', 'Order_Date', 'Ship_Date', 'Ship_Mode', 'Customer_ID',
       'Product_ID', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [76]:
orders.rename(
    columns={
        'Order ID': 'Order_Code',
        'Order Date': 'Order_Date',
        'Ship Date': 'Ship_Date',
        'Ship Mode': 'Ship_Mode',
        'Customer ID': 'Customer_ID',
        'Product ID': 'Product_ID'}, inplace=True)

In [77]:
orders.columns

Index(['Order_Code', 'Order_Date', 'Ship_Date', 'Ship_Mode', 'Customer_ID',
       'Product_ID', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [79]:

from datetime import date


query = []
for index, row in orders.iterrows():
    query += [(
        str(orders.Order_Code[index]),
        (orders.Order_Date[index]),
        (orders.Ship_Date[index]),
        str(orders.Ship_Mode[index]),
        str(orders.Customer_ID[index]),
        str(orders.Product_ID[index]),
        float(orders.Sales[index]),
        int(orders.Quantity[index]),
        float(orders.Discount[index]),
        float(orders.Profit[index])
    )]

In [80]:
cursor.executemany("""
    INSERT INTO orders (
        Order_Code, 
        Order_Date, 
        Ship_Date,
        Ship_Mode,
        Customer_ID,
        Product_ID,
        Sales,
        Quantity,
        Discount,
        Profit
    ) VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10)""", query)

print(cursor.rowcount, "Rows Inserted")


9994 Rows Inserted


In [69]:
connection.commit()

In [51]:
connection.close()